# ImageNet-1K AST Validation - Ultra Configuration

**Developed by Oluwafemi Idiakhoa**

**Goal**: Validate Adaptive Sparse Training on full ImageNet-1K (1.28M images)

**Expected Results**:
- Accuracy: 70-72%
- Energy Savings: 80%
- Training Time: 8 hours on V100

---

## Setup Instructions:

1. **Runtime**: Change to **A100 GPU** (preferred) or **V100**
   - Runtime → Change runtime type → Hardware accelerator: GPU → GPU type: A100

2. **Run all cells** in order

3. **Estimated time**: 5-8 hours depending on GPU

4. **Important**: Mount Google Drive to save checkpoints!


## Step 1: Environment Setup

In [ ]:
# Check GPU
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Install dependencies
!pip install -q torch torchvision tqdm matplotlib

# Clone repository
!git clone https://github.com/oluwafemidiakhoa/adaptive-sparse-training.git
%cd adaptive-sparse-training

print("✅ Dependencies installed and repository cloned!")

## Step 2: Mount Google Drive (For checkpoints)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.makedirs("/content/drive/MyDrive/ast_imagenet1k_checkpoints", exist_ok=True)
print("✅ Google Drive mounted - checkpoints will be saved here")

## Step 3: Dataset Setup

**Choose ONE option below:**

### Option A: Kaggle Dataset (Recommended)

In [ ]:
# Upload kaggle.json first (uncomment to use)
# from google.colab import files
# uploaded = files.upload()  # Upload your kaggle.json

# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

# # Download ImageNet (WARNING: 150GB, takes 2-3 hours!)
# !kaggle competitions download -c imagenet-object-localization-challenge
# !unzip -q imagenet-object-localization-challenge.zip -d /content/imagenet

# data_dir = "/content/imagenet/ILSVRC/Data/CLS-LOC"
# print(f"✅ ImageNet-1K downloaded to: {data_dir}")

### Option B: Google Drive (If you already have ImageNet)

In [ ]:
# Uncomment and set your path
# data_dir = "/content/drive/MyDrive/ImageNet1K/ILSVRC/Data/CLS-LOC"
# print(f"✅ Using ImageNet from Drive: {data_dir}")

### Verify Dataset

In [ ]:
# Set your data_dir here (from one of the options above)
data_dir = "/content/imagenet/ILSVRC/Data/CLS-LOC"  # UPDATE THIS!

import os
if os.path.exists(data_dir):
    train_dir = os.path.join(data_dir, "train")
    val_dir = os.path.join(data_dir, "val")
    
    if os.path.exists(train_dir) and os.path.exists(val_dir):
        num_train_classes = len(os.listdir(train_dir))
        num_val_classes = len(os.listdir(val_dir))
        print(f"✅ Dataset verified!")
        print(f"   Train classes: {num_train_classes}")
        print(f"   Val classes: {num_val_classes}")
    else:
        print(f"❌ Train/Val directories not found in {data_dir}")
else:
    print(f"❌ Dataset directory not found: {data_dir}")
    print("   Please update data_dir variable above")

## Step 4: Load Configuration

In [ ]:
from KAGGLE_IMAGENET1K_AST_CONFIGS import get_config

# Get Ultra configuration (8 hours, 80% energy savings)
config = get_config("ultra")

# Update with your data directory
config.data_dir = data_dir

# Adjust for Colab
config.num_workers = 4  # Colab works better with fewer workers
config.batch_size = 256  # Adjust if OOM errors

print("="*60)
print("ULTRA CONFIGURATION - Quick Validation")
print("="*60)
print(f"Total Epochs: {config.num_epochs}")
print(f"Warmup Epochs: {config.warmup_epochs}")
print(f"Target Activation Rate: {config.target_activation_rate:.0%}")
print(f"Expected Energy Savings: {(1-config.target_activation_rate)*100:.0f}%")
print(f"Batch Size: {config.batch_size}")
print(f"Learning Rate: {config.ast_lr}")
print(f"Estimated Time: 5-8 hours on A100/V100")
print("="*60)

## Step 5: Training Script

**Note**: This will take 5-8 hours. The cell will show progress as it trains.

**Important**: Keep this tab open or it may disconnect!

In [ ]:
print("🚀 Starting ImageNet-1K AST Training...")
print("\nThis will take approximately 5-8 hours.")
print("Checkpoints will be saved to Google Drive every 5 epochs.")
print("\nYou can monitor progress below:")
print("="*60)

# TODO: Import and run the training script
# This requires adapting KAGGLE_IMAGENET100_AST_PRODUCTION.py
# to work with ImageNet-1K (1000 classes instead of 100)

print("\n⚠️ Training script needs to be completed.")
print("Next step: Adapt ImageNet-100 production script for 1000 classes")
print("\nFor now, this notebook provides the setup and configuration.")

## Step 6: Results Analysis (After Training)

In [ ]:
# This cell will analyze results after training completes
# TODO: Add results visualization and analysis

print("Results will be displayed here after training completes")

## 📊 Expected Results

After 8 hours of training with Ultra configuration:

```
Metric                  Value       vs Baseline
Top-1 Accuracy         70-72%       -4 to -6%
Top-5 Accuracy         89-90%       -2 to -3%
Energy Savings         80%          +80%
Training Time          8 hours      6-8× faster
```

**If you achieve 70%+ accuracy → SUCCESS!** ✅

This validates that AST scales to ImageNet-1K (1.28M images)!

## 📝 Next Steps

1. **Document results** - Save final accuracy, energy savings, training time
2. **Update README** - Add ImageNet-1K results section
3. **Announce** - Share results with community
4. **Optional**: Run Conservative config (40h) for publication-quality results

---

**Developed by Oluwafemi Idiakhoa**

GitHub: https://github.com/oluwafemidiakhoa/adaptive-sparse-training